In [ ]:
!pip install tensorflow_addons
#from google.colab import files
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow_addons.metrics import RSquare
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
import os
from keras import backend as K

In [ ]:
%cd Data

In [ ]:
tf.__version__

In [ ]:
temp = []
files = []
for filename in os.listdir():
    if(filename!="Untitled.ipynb" and filename!=".ipynb_checkpoints"):
        print(filename)
        files.append(filename)
        temp.append(pd.read_excel(filename))

In [ ]:
X = []
y = []
for i in range(len(temp)):
  X.append(temp[i]['soc'].to_numpy())
  y.append(temp[i]['Power Loss'].to_numpy())

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
def build_model(i):
  model = Sequential()
  LSTM(1, input_shape=(i+1,1))
  #model.add(Dense(4))
  model.add(Dense(3))
  model.add(Dense(2))
  model.add(Dense(1))
  model.compile(loss='mae', optimizer=Adam(learning_rate = 0.01), metrics=[root_mean_squared_error,RSquare()])
  return model

In [ ]:
%cd ..

In [ ]:
n = [23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75]
scale = [1.6,1.8,2,2.2]
scale = np.array(scale)
shift = np.array(0.5*scale)
best_params = []
for k in range(len(X)):
    for i in range(len(n)):
      for j in range(len(scale)):
        print('running with ',n[i],scale[j])
        X_expanded = np.expand_dims(scale[j]*X[k]-0.5*scale[j], axis=1) 
        poly = PolynomialFeatures(n[i])
        X_expanded = poly.fit_transform(X_expanded)
        model = build_model(n[i])
        s = "best_model_"+files[k][:-5]+"_"+str(n[i])+"_"+str(scale[j])+".hdf5"
        model_save = ModelCheckpoint(s,save_best_only = True)
        model1 = model.fit(X_expanded,y[k],validation_split = 0.2,epochs = 500,callbacks = [model_save])
        best_params.append([n[i],scale[j],max(model1.history['val_r_square']),min(model1.history['val_loss']),min(model1.history['root_mean_squared_error'])])
        plt.plot(model.predict(X_expanded))
        plt.plot(y[k])
        plt.show()
        s1 = "Run_for_file_" + files[k]+".csv"
        best_params_df = pd.DataFrame(best_params)
        best_params_df.to_csv(s1)